<a href="https://colab.research.google.com/github/muziejus/coms-w4995-applied-machine-learning-project/blob/main/notebooks/combine_sentiment_and_financial_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Combine Sentiment and Financial Data

This notebook takes the merged financial dataset prepared by Opel and Meunier along with the sentiment analysis results of each individual company’s articles (with code provided by Mai and Sá Pereira) and generates five individual parquet files, one for each company.

This code was prepared by Sá Pereira and accomplishes these tasks before merging:

1. Aggregates the per-article sentiment data by publication date to produce weighted averages and errors as well as rolling means for sentiment.

2. Breaks apart the financial data back into per-company structures and adds rolling means for price and volume as well as Bollinger bands. Then we add a numerical target (price) and a buy/sell classifaction target (1, 0).

The merged files are on GitHub in `data/` and have the name `<company>_merged_data.parquet`

In [80]:
import pandas as pd

## Set Constants

Here we intialize the constants for loading data, our list of companies, and initialize the financial DataFrame, which has indicators for all five companies as well as external indicators merged together. The columns are broken apart and described below. The variable `sentiment_article_number` pertains to the (maximum) number of articles analyzed. We analyzed them in batches of 25,000 articles at once for each company.

In [81]:
root_data_url = "https://github.com/muziejus/coms-w4995-applied-machine-learning-project/raw/refs/heads/main/data"
financial_data_url = "financial_data"
sentiment_article_number = 75000
sentiment_data_url = f"sentiment_data/analyzed_{sentiment_article_number}"
sentiment_file_name_tail = f"_x_{sentiment_article_number}.parquet"

companies = ["dltr", "lulu", "ulta", "wba", "wmt"]

In [82]:
fin_df = pd.read_csv(f"{root_data_url}/{financial_data_url}/merged_data.csv")

## Create aggregated sentiment data

For each company, we read in the parquet file from GitHub that has the results of our sentiment analysis. The columns in the parquet file are:

Column | Type | Description
---|---|---
`index` | int | A naive index created during aggregation.
`goid` | int | ProQuest’s globally unique identifier for the article in question.
`date`| str | The article’s publication date (`%Y-%m-%d`).
`tokens` | int | A naive (breaking on whitespace) count of tokens in the article.
`corpus` | str | The corpus from which the article comes. Used previously in aggregation.
`daily_article_count` | int | A previously calculated count of all the articles in the corpus from that day.
`daily_token_sum` | int | A sum of all the (naive) tokens from all the articles in the corpus from that day.
`text_sentiment` | float | The overall average sentiment for that article, from (-1, 1) with negative numbers corresponding to negative sentiments and positive with positive.
`text_error` | float | The weighted inverse error in analysis. Higher is more confident in classification.
`text_input_tokens` | int | The number of tokens as analyzed by RoBERTa’s byte-pair encoding tokenizer.

We group the data by date and create a new DataFrame with the following columns:

Column | Type | Description
---|---|---
(index)| datetime | The articles’ publication date.
`analyzed_bpe_tokens`| int | The number of tokens as analyzed by RoBERTa’s BPE tokenizer for all the articles analyzed for the day.
`weighted_sentiment` | float | The mean sentiment for the day (from (-1, 1) as above), weighted by each individual article’s length.
`weighted_error` | float | The mean inverse error for the day, weighted by each individual article’s length. Higher is more confident.
`analyzed_naive_tokens` | int | The number of naive tokens (words separated by whitespace) analyzed for the day.
`daily_naive_token_sum` | int | The total number of naive tokens available for the day.
`analyzed_article_count` | int | The number of articles analyzed for the day.
`daily_article_count` | int | The number of available articles for the day.
`sentiment_3d_rolling_mean` | float | Three-day rolling average for sentiment.
`sentiment_7d_rolling_mean` | float | Seven-day rolling average for sentiment.
`sentiment_14d_rolling_mean` | float | Fourteen-day rolling average for sentiment.

In [83]:
def weighted_avg(row, value_column, weight_column):
    return (row[value_column] * row[weight_column]).sum() / row[weight_column].sum()

def aggregate_sentiment(df):
  agg_sent_df = (
    df.groupby("date")
    .agg(
        analyzed_bpe_tokens = ("text_input_tokens", lambda row: row.sum().astype(int)),
        weighted_sentiment=(
            "text_input_tokens",
            lambda row: weighted_avg(df.loc[row.index], "text_sentiment", "text_input_tokens")
            ),
        weighted_error=(
            "text_input_tokens",
            lambda row: weighted_avg(df.loc[row.index], "text_error", "text_input_tokens")
            ),
        analyzed_naive_tokens = ("tokens", "sum"),
        daily_naive_token_sum = ("daily_token_sum", "first"),
        analyzed_article_count = ("index", "count"),
        daily_article_count = ("daily_article_count", "first")
    )
  )
  agg_sent_df.reset_index(inplace=True)
  agg_sent_df["date"] = pd.to_datetime(agg_sent_df.date)
  agg_sent_df.sort_values("date", inplace=True) # probably redundant
  agg_sent_df.set_index('date', inplace=True)

  for window in [3, 7, 14]:
    agg_sent_df[f"sentiment_{window}d_rolling_mean"] = agg_sent_df.weighted_sentiment.rolling(window).mean()

  return agg_sent_df

## Create Company-Specific Financial Data

For each company, we use the financial dataframe to create a new data frame specific to the company. It has these columns:

Column | Type | Description
---|---|---
(index)| datetime | The date.
`open` | float | The opening price.
`high` | float | The high price.
`low` | float | The low price.
`close` | float | The closing price.
`volume` | float | The number of shares traded.
`dividends` | float | Dividends paid (if any).
`stock_splits` | float | Stock splits (if any).
`cpi` | float| Consumer Price Index for all urban consumers. Not seasonally adjusted
`pce` | float| Personal Consumption Expenditures index.
`ppi` | float| Producer Price Index for All Commodities.
`eci` | float| Employment Cost Index for All Civilian Workers.
`gdp` |float | GDP Deflator. Adjusts nominal GDP to real GDP.
`unemployment` |float | Unemployment Rate.
`manufacturing` | float | Manufacturing and Construction Employment.
`sp500` | float | Standard & Poor’s 500 Index.
`price_3d_rolling_mean` | float | Rolling three-day price mean.
`price_3d_rolling_std` | float | Rolling three-day price standard deviation.
`volume_3d_rolling_mean` | float | Rolling three-day volume mean.
`volume_3d_rolling_std` | float | Rolling three-day volume standard deviation.
`bollinger_3d_upper_band` | float | Bollinger three-day upper band.
`bollinger_3d_lower_band` | float | Bollinger three-day lower band.
`price_7d_rolling_mean` | float | Rolling seven-day price mean.
`price_7d_rolling_std` | float | Rolling seven-day price standard deviation.
`volume_7d_rolling_mean` | float | Rolling seven-day volume mean.
`volume_7d_rolling_std` | float | Rolling seven-day volume standard deviation.
`bollinger_7d_upper_band` | float | Bollinger seven-day upper band.
`bollinger_7d_lower_band` | float | Bollinger seven-day lower band.
`price_14d_rolling_mean` | float | Rolling 14-day price mean.
`price_14d_rolling_std` | float | Rolling 14-day price standard deviation.
`volume_14d_rolling_mean` | float | Rolling 14-day volume mean.
`volume_14d_rolling_std` | float | Rolling 14-day volume standard deviation.
`bollinger_14d_upper_band` | float | Bollinger 14-day upper band.
`bollinger_14d_lower_band` | float | Bollinger 14-day lower band.
`target_price` | float | Next day’s price.
`target` | int | Buy or sell recommendation (1 or 0)


In [84]:
def create_company_financial_data(company, fin_df):
  financial_columns = [
      "Date",
      f"Open_{company}",
      f"High_{company}",
      f"Low_{company}",
      f"Close_{company}",
      f"Volume_{company}",
      f"Dividends_{company}",
      f"Stock Splits_{company}",
      "CPIAUCSL", "PCE",
      "PPIACO", "ECIALLCIV", "GDPDEF", "UNRATE", "MCUMFN", "SP500"
  ]
  new_column_names = {
      "Date": "date",
      f"Open_{company}": "open",
      f"High_{company}": "high",
      f"Low_{company}": "low",
      f"Close_{company}": "close",
      f"Volume_{company}": "volume",
      f"Dividends_{company}": "dividends",
      f"Stock Splits_{company}": "stock_splits",
      "CPIAUCSL": "cpi",
      "PCE": "pce",
      "PPIACO": "ppi",
      "ECIALLCIV": "eci",
      "GDPDEF": "gdp",
      "UNRATE": "unemployment",
      "MCUMFN": "manufacturing",
      "SP500": "sp500"
  }
  company_fin_df = fin_df[financial_columns]
  company_fin_df = company_fin_df.rename(columns=new_column_names)

  company_fin_df["date"] = pd.to_datetime(company_fin_df["date"])
  company_fin_df.set_index("date", inplace=True)

  for window in [3, 7, 14]:
    company_fin_df[f"price_{window}d_rolling_mean"] = company_fin_df.close.rolling(window).mean()
    company_fin_df[f"price_{window}d_rolling_std"] = company_fin_df.close.rolling(window).std()
    company_fin_df[f"volume_{window}d_rolling_mean"] = company_fin_df.volume.rolling(window).mean()
    company_fin_df[f"volume_{window}d_rolling_std"] = company_fin_df.volume.rolling(window).std()
    # Bollinger Bands
    std_dev = 2
    company_fin_df[f"bollinger_{window}d_upper_band"] = (
        company_fin_df[f"price_{window}d_rolling_mean"] + (company_fin_df[f"price_{window}d_rolling_std"] * std_dev)
    )
    company_fin_df[f"bollinger_{window}d_lower_band"] = (
        company_fin_df[f"price_{window}d_rolling_mean"] - (company_fin_df[f"price_{window}d_rolling_std"] * std_dev)
    )

  company_fin_df["target_price"] = company_fin_df.close.shift(-1)
  company_fin_df["target"] = (company_fin_df.close.shift(-1) > company_fin_df.close).astype(int)

  return company_fin_df


## Iterate over All Five Companies

We create a DataFrame for each company that merges the sentiment and financial information and then save it as a parquet file for subsequent committing to GitHub.

In [85]:
for company in companies:
  sent_df = pd.read_parquet(f"{root_data_url}/{sentiment_data_url}/{company}{sentiment_file_name_tail}")
  agg_sent_df = aggregate_sentiment(sent_df)
  company_fin_df = create_company_financial_data(company, fin_df)
  merged_df = pd.merge(agg_sent_df, company_fin_df, left_index=True, right_index=True)
  merged_df.to_parquet(f"{company}_merged_data.parquet")